In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import nltk
import pandas as pd
import ast
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
movies = pd.read_csv(r"D:\PYTHON\Projects\Movie-Recommendation-System\movies.csv")
credits = pd.read_csv(r"D:\PYTHON\Projects\Movie-Recommendation-System\credits.csv")

In [9]:
movies = movies.merge(credits, on="title")

In [22]:
movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]
movies.dropna(inplace=True)

In [27]:
def convert(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i["name"])
    return l

In [24]:
def convert_cast(text):
    l = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            l.append(i['name'])
        counter += 1
    return l

In [25]:
def fetch_director(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [28]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert_cast)
movies['crew'] = movies['crew'].apply(fetch_director)

In [29]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [30]:
def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(" ", ''))
    return l

In [31]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [32]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [33]:
new_df = movies[['movie_id', 'title', 'tags']]

In [34]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\priya\AppData\Local\Temp\ipykernel_10548\2959733794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
C:\Users\priya\AppData\Local\Temp\ipykernel_10548\2959733794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [37]:
ps = PorterStemmer()
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [38]:
new_df['tags'] = new_df['tags'].apply(stems)

C:\Users\priya\AppData\Local\Temp\ipykernel_10548\3973021881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stems)


In [39]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [40]:
similarity = cosine_similarity(vector)

NameError: name 'vector' is not defined

In [41]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1: 6]:
        print(new_df.loc[i[0]].title)

In [42]:
recommend('Spider-Man')

NameError: name 'similarity' is not defined